In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import functools
import random
import time
import random
from pygame_visualization import VRPVisualizator

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
vizu = VRPVisualizator()

In [2]:
def manhattan(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [3]:
def generate_coordinates(n_nodes: int, x_coords_range, y_coords_range, min_distance: float):
    generated_coords = [(0, 0)]
    vizu.add_node(0, 0)
    for i in range(n_nodes - 1):
        is_ok = False
        while not is_ok:
            coord = (random.uniform(*x_coords_range), random.uniform(*y_coords_range))
            is_ok = True
            for node in generated_coords:
                if manhattan(coord, node) < min_distance:
                    is_ok = False
        generated_coords.append(coord)
        vizu.add_node(coord[0], coord[1])
    return generated_coords

In [4]:
def generate_graph_matrix(length: int):
    coords = generate_coordinates(length, (-600, 600), (-400, 400), 100)
    matrix = np.empty((length, length))
    matrix[:] = np.nan
    for i in range(length):
        for j in range(length):
            if i != j:
                dist = manhattan(coords[i], coords[j])
                matrix[i][j] = dist
                matrix[j][i] = dist
    return matrix

In [6]:
vizu.clear_nodes()
graph = generate_graph_matrix(20)

In [7]:
vizu.set_scaling(1.3)

In [8]:
print(graph)

[[          nan  233.58963182  592.21298493  320.44367518  347.99977606
   682.90280612  509.15479166  597.27759789  127.07813415  698.1059641
   431.18105904  704.82711797  724.49205179  103.94582426  319.17581109
   433.44176712  500.83597188  308.27313502  176.8273828   459.5995573 ]
 [ 233.58963182           nan  482.39763625  430.25902387  581.58940788
   916.49243794  742.74442348  830.86722971  106.51149767  588.29061541
   540.99640773  938.4167498   614.6767031   302.5181944   505.89776417
   543.25711581  734.4256037   418.08848371  296.89317576  226.00992548]
 [ 592.21298493  482.39763625           nan  912.65666012  903.31761819
  1238.22064825 1064.47263379 1152.59544002  533.19183574  628.20476893
  1023.39404397 1260.1449601   185.11469067  696.1588092   827.62597447
  1025.65475205 1056.15381401  900.48611995  618.62138606  478.32161102]
 [ 320.44367518  430.25902387  912.65666012           nan  465.61723842
   796.29474884  622.54673438  710.66954062  379.46482438 1018

In [9]:
#Evaporation factor for global update of pheromones
RHO = 0.2
#Evaporation factor for local update of pheromones
KAPPA = RHO
#Q
OMICRON = 1
#Impact of pheromones
ALPHA = 1
#Impact of weights
BETA = 2
#Initial pheromones
TAU = 1
#Exploration/exploitation trade off
EPSILON = 0.5

In [10]:
class Ant():
    def __init__(self, init_pos: int, vizualizator: VRPVisualizator) -> None:
        self.init_pos = init_pos
        self.vizualizator = vizualizator
        self.current_pos = init_pos
        self.path_history = [init_pos]
        self.can_continue = True

    def add_to_path_history(self, node) -> None:
        self.path_history.append(node)
        self.vizualizator.set_path(self.path_history)

    def move(self, dest: list, pheromones: list) -> None:
        choice = random.uniform(0, 1)

        if choice <= EPSILON:
            dest_picked = self.exploitation(dest, pheromones)
        else:
            dest_picked = self.biased_exploration(dest, pheromones)

        if not dest_picked:
            self.add_to_path_history(self.init_pos)
            self.can_continue = False
            return

        self.update_pheromones_locally(pheromones, dest_picked)
        self.current_pos = dest_picked
        self.add_to_path_history(self.current_pos)

    def exploitation(self, dest: list, pheromones: list) -> int:
        current_best_viability = None
        current_best_dest = None
        for i in range(len(dest)):
            if not np.isnan(dest[i]) and i not in self.path_history:
                viability = pheromones[i] * (KAPPA / dest[i])**BETA
                if not current_best_viability or viability > current_best_viability:
                    current_best_viability = viability
                    current_best_dest = i
        return current_best_dest

    def biased_exploration(self, dest: list, pheromones: list) -> int:
        probabilities = []
        denominator = 0
        
        #Calculate the denominator first
        for i in range(len(dest)):
            if not np.isnan(dest[i]) and i not in self.path_history:
                denominator += pheromones[i]**ALPHA * (KAPPA/dest[i])**BETA

        #Calculate probabilities of picking one path
        for node, length in enumerate(dest):
            if node in self.path_history:
                probabilities.append(0)
            elif not np.isnan(dest[node]):
                nominator = pheromones[node]**ALPHA * (KAPPA/dest[node])**BETA
                probabilities.append(nominator / denominator)
            else:
                probabilities.append(np.nan)
        
        #If there is no path available return false
        if np.nansum(probabilities) == 0:
            return None

        #Roulette wheel
        cumulative_sum = []
        for i in range(len(probabilities)):
            if np.isnan(probabilities[i]):
                cumulative_sum.append(np.nan)
            elif np.nansum(cumulative_sum) == 0:
                cumulative_sum.append(1.0)
            else:
                cumulative_sum.append(np.nansum(probabilities[i:len(probabilities)]))
        
        #Pick a destination
        rand = random.uniform(0, 1)
        dest_picked = None
        cumulative_sum.append(0)

        for i in range(0, len(cumulative_sum)-1):
            p = cumulative_sum[i]
            nextp = cumulative_sum[i+1] if not np.isnan(cumulative_sum[i+1]) else cumulative_sum[i+2] if not np.isnan(cumulative_sum[i+2]) else cumulative_sum[i+3]
        
            if not np.isnan(p) and rand <= p and rand >= nextp:
                dest_picked = i
        return dest_picked

    def update_pheromones_locally(self, pheromones: list, dest: int) -> None:
        #Apply the ACS local updating rule
        pheromones[dest] = (1-RHO) * pheromones[dest] + RHO * TAU


In [11]:
class ACO():
    def __init__(self, graph, start: int, vizualizator: VRPVisualizator) -> None:
        self.graph = graph
        self.start = start
        self.vizualizator = vizualizator
        self.current_best_path = None
        self.current_best_length = None
        self.pheromones = np.ones(graph.shape)
        #self.vizualizator.set_pheromones(self.pheromones)

    def run(self, iter: int) -> list:
        for i in range(iter):
            print('Tour:', i)
            self.tour_construction(15)
            self.global_update_pheromones()
        self.vizualizator.set_path(self.current_best_path)
        print('Best:', self.current_best_path)
        print('Length:', self.current_best_length)
        return self.current_best_path

    def tour_construction(self, ant_amount: int) -> None:
        ants = [Ant(self.start, self.vizualizator) for i in range(ant_amount)]
        while ants:
            for ant in ants:
                if ant.can_continue:
                    ant.move(self.graph[ant.current_pos], self.pheromones[ant.current_pos])
                else:
                    self.update_current_best(ant.path_history)
                    ants.remove(ant)

    def update_current_best(self, path):
        path_length = self.calc_total_distance(path)
        if not self.current_best_path or path_length < self.current_best_length:
            print('NEW BEST:', path)
            self.current_best_path = path
            self.current_best_length = path_length

    def global_update_pheromones(self) -> None:
        #Evaporation
        self.pheromones *= 1 - RHO
        #New pheromones
        for i in range(len(self.current_best_path)-1):
            current_node = self.current_best_path[i]
            next_node = self.current_best_path[i+1]
            self.pheromones[current_node][next_node] += RHO * (KAPPA / self.current_best_length)
           #self.pheromones[next_node][current_node] += RHO * (KAPPA / self.current_best_length)
        #self.vizualizator.set_pheromones(self.pheromones)

    def calc_total_distance(self, full_path: list) -> float:
        distance = 0
        for i in range(len(full_path)-1):
            cur_path = full_path[i]
            next_path = full_path[i+1]
            distance += self.graph[cur_path][next_path]
        return distance


In [12]:
aco = ACO(graph, 0, vizu)
start = time.time()
path = aco.run(40)
print(path)
print(time.time()-start)

Tour: 0
NEW BEST: [0, 1, 9, 13, 3, 10, 16, 15, 19, 4, 14, 6, 12, 11, 2, 17, 5, 8, 18, 7, 0]
NEW BEST: [0, 1, 9, 19, 4, 14, 18, 7, 3, 10, 13, 15, 16, 12, 11, 2, 5, 8, 17, 6, 0]
NEW BEST: [0, 1, 9, 8, 5, 2, 17, 12, 19, 4, 14, 18, 7, 3, 13, 10, 15, 16, 6, 11, 0]
Tour: 1
Tour: 2
NEW BEST: [0, 4, 14, 19, 9, 11, 12, 15, 10, 13, 16, 6, 2, 17, 5, 8, 1, 3, 18, 7, 0]
NEW BEST: [0, 1, 9, 19, 4, 14, 2, 17, 5, 8, 7, 18, 3, 10, 13, 15, 16, 6, 12, 11, 0]
Tour: 3
Tour: 4
Tour: 5
NEW BEST: [0, 1, 9, 19, 4, 14, 18, 7, 3, 10, 13, 15, 16, 6, 12, 11, 2, 17, 5, 8, 0]
Tour: 6
Tour: 7
NEW BEST: [0, 1, 9, 2, 17, 5, 8, 19, 4, 14, 18, 7, 3, 15, 13, 10, 16, 6, 12, 11, 0]
Tour: 8
Tour: 9
Tour: 10
Tour: 11
Tour: 12
Tour: 13
Tour: 14
NEW BEST: [0, 1, 9, 2, 17, 5, 8, 19, 4, 14, 18, 7, 3, 10, 13, 15, 16, 6, 12, 11, 0]
Tour: 15
Tour: 16
Tour: 17
Tour: 18
Tour: 19
Tour: 20
Tour: 21
Tour: 22
Tour: 23
Tour: 24
Tour: 25
Tour: 26
Tour: 27
Tour: 28
Tour: 29
Tour: 30
Tour: 31
Tour: 32
Tour: 33
Tour: 34
Tour: 35
Tour: 36
Tour: